In [1]:
import sys
assert sys.version_info[0] >= 3

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

## Init auth log data

In [2]:
%matplotlib inline
with open('C:/Users/Sotiris/Desktop/project/data/auth.log') as f:
    init_data = f.read().split('\n')

In [3]:
example = [x for x in init_data if 'user=root' in x][0]
example

'Mar 29 23:18:19 ip-10-77-20-248 sshd[6402]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=218.60.136.106  user=root'

In [4]:
'Mar 29 19:30:18 ip-10-77-20-248 sshd[5743]: Failed password for elastic_user_4 from 24.151.103.17 port 52124 ssh2'

'Mar 29 19:30:18 ip-10-77-20-248 sshd[5743]: Failed password for elastic_user_4 from 24.151.103.17 port 52124 ssh2'

## Create logs template 

In [5]:
template = '{:%b %d %H:%M:%S} {} sshd[1234]: Failed password for {} from {} port 1234 ssh2'

In [6]:
d = datetime(2018,3,10,10,45,10)
hostname = 'my-host-1-2-3-4'
ip = '192.168.10.1'
user = 'lab_user_1'

In [7]:
print(d,hostname,ip,user)

2018-03-10 10:45:10 my-host-1-2-3-4 192.168.10.1 lab_user_1


In [99]:
template.format(d,hostname,user,ip)

'Mar 10 10:45:10 my-host-1-2-3-4 sshd[1234]: Failed password for lab_user_1 from 192.168.10.1 port 1234 ssh2'

##  Create time index generator and df 

In [24]:
def create_time_index(start_date='22/3/2018 12:43:23',
                      end_date='30/3/2018',
                      period_minutes=120, #period in minutes
                      noise_minutes=10): #noise in minutes
    
    period_minutes=int(period_minutes)
    assert period_minutes>0
    
    noise_minutes = int(noise_minutes)
    # assert noise_minutes>0

    dates = pd.date_range(start_date,end_date,freq='{}T'.format(period_minutes))
    dates += pd.Series([pd.Timedelta(str(int(x))+'m') for x in np.random.normal(0, noise_minutes, dates.shape)])
    return dates

In [10]:
def create_df(time_index, host='my-host-1', ip='192.168.0.1', user='lab_user1'):
    df = pd.DataFrame([[host,ip,user] for _ in time_index],
                      columns=['host','ip','user'],
                      index=time_index)
    return df

### Examples

In [11]:
create_time_index('1/27/2016 14:23:23','1/28/2016 16:23:23',5,1)

0     2016-01-27 14:23:23
1     2016-01-27 14:28:23
2     2016-01-27 14:34:23
3     2016-01-27 14:38:23
4     2016-01-27 14:43:23
5     2016-01-27 14:47:23
6     2016-01-27 14:53:23
7     2016-01-27 14:58:23
8     2016-01-27 15:05:23
9     2016-01-27 15:08:23
10    2016-01-27 15:13:23
11    2016-01-27 15:18:23
12    2016-01-27 15:23:23
13    2016-01-27 15:28:23
14    2016-01-27 15:33:23
15    2016-01-27 15:38:23
16    2016-01-27 15:42:23
17    2016-01-27 15:49:23
18    2016-01-27 15:52:23
19    2016-01-27 15:58:23
20    2016-01-27 16:03:23
21    2016-01-27 16:09:23
22    2016-01-27 16:13:23
23    2016-01-27 16:19:23
24    2016-01-27 16:22:23
25    2016-01-27 16:28:23
26    2016-01-27 16:33:23
27    2016-01-27 16:39:23
28    2016-01-27 16:43:23
29    2016-01-27 16:48:23
              ...        
283   2016-01-28 13:58:23
284   2016-01-28 14:03:23
285   2016-01-28 14:09:23
286   2016-01-28 14:13:23
287   2016-01-28 14:18:23
288   2016-01-28 14:23:23
289   2016-01-28 14:26:23
290   2016-0

In [12]:
every_24h_noisy = create_time_index('2/1/2016','2/14/2016',24*60,5*60)
df1 = create_df(every_24h_noisy,ip='192.168.0.24',user='lab_user_A')

every_24h = create_time_index('2/1/2016','2/14/2016',24*60,10)
df2 = create_df(every_24h,ip='192.168.0.2',user='lab_user_B')

every_12h = create_time_index('2/1/2016','2/14/2016',12*60,60)
df3 = create_df(every_12h,ip='192.168.0.10',user='lab_user_C')

every_1h = create_time_index('2/1/2016','2/14/2016',1*60,10)
df4 = create_df(every_1h,ip='192.168.0.124',user='lab_user_A')

### Concat data and plot

In [13]:
all_dfs = pd.concat([df1,df2,df3,df4])
all_dfs.sort_index(inplace=True)
all_dfs.head()

,host,ip,user
2016-01-31 23:36:00,my-host-1,192.168.0.10,lab_user_C
2016-01-31 23:50:00,my-host-1,192.168.0.2,lab_user_B
2016-01-31 23:55:00,my-host-1,192.168.0.124,lab_user_A
2016-02-01 00:56:00,my-host-1,192.168.0.124,lab_user_A
2016-02-01 02:10:00,my-host-1,192.168.0.124,lab_user_A


In [14]:
all_dfs.values

array([['my-host-1', '192.168.0.10', 'lab_user_C'],
       ['my-host-1', '192.168.0.2', 'lab_user_B'],
       ['my-host-1', '192.168.0.124', 'lab_user_A'],
       ..., 
       ['my-host-1', '192.168.0.2', 'lab_user_B'],
       ['my-host-1', '192.168.0.10', 'lab_user_C'],
       ['my-host-1', '192.168.0.24', 'lab_user_A']], dtype=object)

In [ ]:
plt.plot_date(all_dfs.index,[' '.join(x) for x in all_dfs.values])

## Generate Examples with anomalies

In [16]:
anomalus_df = create_time_index('2/5/2016 12:00:00','2/6/2016 12:00:00',period_minutes=3*60,noise_minutes=20)
# same info as the 'every 24h df'
df_an = create_df(anomalus_df,ip='192.168.0.2',user='lab_user_B')

In [17]:
all_dfs = pd.concat([all_dfs,df_an])
all_dfs.sort_index(inplace=True)
all_dfs.head()

,host,ip,user
2016-01-31 23:36:00,my-host-1,192.168.0.10,lab_user_C
2016-01-31 23:50:00,my-host-1,192.168.0.2,lab_user_B
2016-01-31 23:55:00,my-host-1,192.168.0.124,lab_user_A
2016-02-01 00:56:00,my-host-1,192.168.0.124,lab_user_A
2016-02-01 02:10:00,my-host-1,192.168.0.124,lab_user_A


In [ ]:
plt.plot_date(all_dfs.index,[' '.join(x) for x in all_dfs.values])

## Everything under one function

In [25]:
def generate_events(start_timedate_us_format ='3/22/2016 12:00:23',
                   end_timedate_us_format = '3/28/2016 21:12:43',
                   period_in_minutes=3*60,
                   noise_std_in_minutes=20,
                   host='my-host-1',
                   ip='192.168.0.1',
                   user='test_lab_user'
                  ):
    df_index = create_time_index(start_date = start_timedate_us_format,
                                 end_date = end_timedate_us_format,
                                 period_minutes = period_in_minutes,
                                 noise_minutes = noise_std_in_minutes)
    
    return_df = create_df(df_index,
                          host=host,
                          ip=ip,
                          user=user)    
        
    return return_df

def generate_events_from_list(list_of_settings):
    dfs_list = [generate_events(*settings) for settings in list_of_settings]
    
    all_dfs = pd.concat(dfs_list)
    all_dfs.sort_index(inplace=True)
    
    return(all_dfs)

def plot_dates_categorical(dfs):
    return plt.plot_date(dfs.index,[' '.join(x) for x in dfs.values])

In [20]:
settings = [
    ('1/1/2016', '1/30/2016',24*60,60,'ip-10-77-20-248','10.10.24.1','lab_user1'),
    ('1/12/2016','1/15/2016',3*60,20,'ip-10-77-20-248', '10.10.24.1','lab_user1'),
    ('1/25/2016','1/27/2016',1*60,20,'ip-10-77-20-248', '10.10.24.1','lab_user1'),
    ('1/12/2016','1/15/2016',1*60,10,'ip-10-77-20-248', '10.10.1.10','lab_user2'),
    ('1/5/2016', '1/17/2016',20,   2,'ip-10-77-20-248', '10.10.1.1', 'lab_user2'),
    ('1/2/2016', '1/28/2016',12*60,30,'ip-10-77-20-248','10.10.1.12','lab_user3'),
    ('1/4/2016', '1/5/2016',2*60,10,'ip-10-77-20-248',  '10.10.1.12','lab_user3')]

In [45]:
settings = [
    ('1/1/2016 12:00:00',  '1/30/2016',24*60, 0,'ip-10-77-20-248','10.10.24.1','lab_user1'),
    ('1/1/2016 9:00:00',   '1/30/2016',24*60, 0,'ip-10-77-20-248','10.10.24.1','lab_user1'),
    ('1/10/2016 15:00:00', '1/20/2016',24*60, 0,'ip-10-77-20-248','10.10.24.1','lab_user1'),
    ('1/10/2016 15:02:00', '1/20/2016',24*60, 0,'ip-10-77-20-248','10.10.24.1','lab_user1'),
    ('1/10/2016 15:04:00', '1/20/2016',24*60, 0,'ip-10-77-20-248','10.10.24.1','lab_user1'),
    ('1/10/2016 15:06:00', '1/20/2016',24*60, 0,'ip-10-77-20-248','10.10.24.1','lab_user1'),
    ('1/10/2016 15:08:00', '1/20/2016',24*60, 0,'ip-10-77-20-248','10.10.24.1','lab_user1')]

In [31]:
import random

def all_day_many_failures(number_of_failures):
    """
    :param number_of_failures: failures each day between 8 and 24 hours
    """
    hours = list(range(8,24))
    minutes = list(range(60))
    
    dates = []
    for i in range(1,30):
        for _ in range(number_of_failures):
            hour = random.choice(hours)
            minute = random.choice(minutes)
        
            dates.append(datetime(2018,1,i,hour,minute,0))
            #dates.append('2016-01-' + day + ' ' + hour + ':' + minute + ':00')
            
    return dates
            
dates1 = all_day_many_failures(10)
dates2 = all_day_many_failures(10)
dates3 = all_day_many_failures(10)
dates4 = all_day_many_failures(10)
dates5 = all_day_many_failures(10)
dates6 = all_day_many_failures(10)
dates7 = all_day_many_failures(10)
dates8 = all_day_many_failures(10)
dates9 = all_day_many_failures(10)
dates10 = all_day_many_failures(10)
dates11 = all_day_many_failures(10)
dates12 = all_day_many_failures(10)
dates13 = all_day_many_failures(10)
dates14 = all_day_many_failures(10)
dates15 = all_day_many_failures(10)

In [32]:
df1 = create_df(dates1,ip='192.168.0.1',user='user_normal_A', host='ip-10-77-20-248')
df1.index = pd.to_datetime(df1.index)
df2 = create_df(dates2,ip='192.168.0.2',user='user_normal_B', host='ip-10-77-20-248')
df2.index = pd.to_datetime(df2.index)
df3 = create_df(dates3,ip='192.168.0.3',user='user_normal_C', host='ip-10-77-20-248')
df3.index = pd.to_datetime(df3.index)
df4 = create_df(dates4,ip='192.168.0.4',user='user_normal_D', host='ip-10-77-20-248')
df4.index = pd.to_datetime(df4.index)
df5 = create_df(dates5,ip='192.168.0.5',user='user_normal_E', host='ip-10-77-20-248')
df5.index = pd.to_datetime(df5.index)
df6 = create_df(dates6,ip='192.168.0.5',user='user_normal_F', host='ip-10-77-20-248')
df6.index = pd.to_datetime(df6.index)
df7 = create_df(dates7,ip='192.168.0.7',user='user_normal_G', host='ip-10-77-20-248')
df7.index = pd.to_datetime(df7.index)
df8 = create_df(dates8,ip='192.168.0.8',user='user_normal_H', host='ip-10-77-20-248')
df8.index = pd.to_datetime(df8.index)
df9 = create_df(dates9,ip='192.168.0.9',user='user_normal_I', host='ip-10-77-20-248')
df9.index = pd.to_datetime(df9.index)
df10 = create_df(dates10,ip='192.168.0.10',user='user_normal_G', host='ip-10-77-20-248')
df10.index = pd.to_datetime(df10.index)
df11 = create_df(dates11,ip='192.168.0.11',user='user_normal_K', host='ip-10-77-20-248')
df11.index = pd.to_datetime(df11.index)
df12 = create_df(dates12,ip='192.168.0.12',user='user_normal_L', host='ip-10-77-20-248')
df12.index = pd.to_datetime(df12.index)
df13 = create_df(dates13,ip='192.168.0.13',user='user_normal_M', host='ip-10-77-20-248')
df13.index = pd.to_datetime(df13.index)
df14 = create_df(dates14,ip='192.168.0.14',user='user_normal_N', host='ip-10-77-20-248')
df14.index = pd.to_datetime(df14.index)
df15 = create_df(dates15,ip='192.168.0.15',user='user_normal_O', host='ip-10-77-20-248')
df15.index = pd.to_datetime(df15.index)

In [33]:
all_dfs = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15])

In [34]:
all_dfs[:7]

,host,ip,user
2018-01-01 10:16:00,ip-10-77-20-248,192.168.0.1,user_normal_A
2018-01-01 18:37:00,ip-10-77-20-248,192.168.0.1,user_normal_A
2018-01-01 08:37:00,ip-10-77-20-248,192.168.0.1,user_normal_A
2018-01-01 20:00:00,ip-10-77-20-248,192.168.0.1,user_normal_A
2018-01-01 20:19:00,ip-10-77-20-248,192.168.0.1,user_normal_A
2018-01-01 19:07:00,ip-10-77-20-248,192.168.0.1,user_normal_A
2018-01-01 11:58:00,ip-10-77-20-248,192.168.0.1,user_normal_A


In [35]:
save_as_log_file('C:/Users/Sotiris/Desktop/project/data/tests/fails_during_day_150_by_15_users.log',all_dfs)

Saved to C:/Users/Sotiris/Desktop/project/data/tests/fails_during_day_150_by_15_users.log
Done


In [46]:
all_data = generate_events_from_list(settings)

In [ ]:
plot_dates_categorical(all_data)

In [47]:
save_as_log_file('C:/Users/Sotiris/Desktop/project/data/tests/single_user_two_fails_every_day_after_10_days_5_fails_without_noise.log',all_data)

Saved to C:/Users/Sotiris/Desktop/project/data/tests/single_user_two_fails_every_day_after_10_days_for_10_days_5_fails_every_day_same_time_without_noise.log
Done


## Export as logs

In [43]:
temp_data = all_data

In [44]:
temp_data

,host,ip,user
2016-01-01 00:19:00,ip-10-77-20-248,10.10.24.1,lab_user1
2016-01-02 00:13:00,ip-10-77-20-248,10.10.1.12,lab_user3
2016-01-02 00:54:00,ip-10-77-20-248,10.10.24.1,lab_user1
2016-01-02 12:24:00,ip-10-77-20-248,10.10.1.12,lab_user3
2016-01-02 23:16:00,ip-10-77-20-248,10.10.24.1,lab_user1
2016-01-03 00:06:00,ip-10-77-20-248,10.10.1.12,lab_user3
2016-01-03 11:38:00,ip-10-77-20-248,10.10.1.12,lab_user3
2016-01-04 00:06:00,ip-10-77-20-248,10.10.1.12,lab_user3
2016-01-04 00:34:00,ip-10-77-20-248,10.10.1.12,lab_user3
2016-01-04 01:37:00,ip-10-77-20-248,10.10.24.1,lab_user1


In [45]:
template = '{:%b %d %H:%M:%S} {} sshd[1234]: Failed password for {} from {} port 1234 ssh2'

In [46]:
for row in temp_data.iterrows():
    print(template.format(row[0],row[1]['host'],row[1]['user'],row[1]['ip']))
    print('---')

Jan 01 00:19:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user1 from 10.10.24.1 port 1234 ssh2
---
Jan 02 00:13:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user3 from 10.10.1.12 port 1234 ssh2
---
Jan 02 00:54:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user1 from 10.10.24.1 port 1234 ssh2
---
Jan 02 12:24:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user3 from 10.10.1.12 port 1234 ssh2
---
Jan 02 23:16:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user1 from 10.10.24.1 port 1234 ssh2
---
Jan 03 00:06:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user3 from 10.10.1.12 port 1234 ssh2
---
Jan 03 11:38:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user3 from 10.10.1.12 port 1234 ssh2
---
Jan 04 00:06:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user3 from 10.10.1.12 port 1234 ssh2
---
Jan 04 00:34:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user3 from 10.10.1.12 port 1234 ssh2
---
Jan 04 01:37:00 ip-

Jan 14 08:20:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user2 from 10.10.1.1 port 1234 ssh2
---
Jan 14 08:39:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user2 from 10.10.1.1 port 1234 ssh2
---
Jan 14 08:50:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user2 from 10.10.1.10 port 1234 ssh2
---
Jan 14 08:50:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user1 from 10.10.24.1 port 1234 ssh2
---
Jan 14 08:59:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user2 from 10.10.1.1 port 1234 ssh2
---
Jan 14 09:22:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user2 from 10.10.1.1 port 1234 ssh2
---
Jan 14 09:43:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user2 from 10.10.1.1 port 1234 ssh2
---
Jan 14 10:01:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user2 from 10.10.1.1 port 1234 ssh2
---
Jan 14 10:02:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user2 from 10.10.1.10 port 1234 ssh2
---
Jan 14 10:21:00 ip-10-77-

In [47]:
logs = []
for row in temp_data.iterrows():
    logs.append(template.format(row[0],row[1]['host'],row[1]['user'],row[1]['ip']))

In [48]:
logs

['Jan 01 00:19:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user1 from 10.10.24.1 port 1234 ssh2',
 'Jan 02 00:13:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user3 from 10.10.1.12 port 1234 ssh2',
 'Jan 02 00:54:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user1 from 10.10.24.1 port 1234 ssh2',
 'Jan 02 12:24:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user3 from 10.10.1.12 port 1234 ssh2',
 'Jan 02 23:16:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user1 from 10.10.24.1 port 1234 ssh2',
 'Jan 03 00:06:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user3 from 10.10.1.12 port 1234 ssh2',
 'Jan 03 11:38:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user3 from 10.10.1.12 port 1234 ssh2',
 'Jan 04 00:06:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user3 from 10.10.1.12 port 1234 ssh2',
 'Jan 04 00:34:00 ip-10-77-20-248 sshd[1234]: Failed password for lab_user3 from 10.10.1.12 port 1234 ssh2',
 'Jan 04 01:37:00 i

In [17]:
def save_as_log_file(filename='temp_logs.log',dataframe=None):
    logs = []
    for row in dataframe.iterrows():
        logs.append(template.format(row[0],row[1]['host'],row[1]['user'],row[1]['ip']))    
    
    with open(filename,'w') as f:
        f.writelines([l+'\n' for l in logs])

    print('Saved to '+filename)
    print('Done')
    

In [50]:
save_as_log_file('C:/Users/Sotiris/Desktop/project/data/generatedData.log',temp_data)

Saved to C:/Users/Sotiris/Desktop/project/data/generatedData.log
Done


###  Save the generated usecases

In [51]:
save_as_log_file('C:/Users/Sotiris/Desktop/project/data/4cases.log',all_data)

Saved to C:/Users/Sotiris/Desktop/project/data/4cases.log
Done
